# Цель: предсказать стоимость комнаты 18 м2 в Царицыно в 2шке
8 мин от метро пешком, дом панельный 16этажка, не элитный, квартира расположена на 15 этаже.

#### для этого с сайта авито/яндекс недвижимость код должен скачивать и парсить данные, превращая в датафрейм, где существуют данные широты и долготы комнаты, ближайшая станция метро, квадратура комнаты, количество комнат в квартире, стоимость комнаты, на каком этаже расположена и сколько всего этажей в доме, тип дома и тип контрагента(посредник/прямой)

In [1865]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import io
import requests
import seaborn as sns
import statistics

from sklearn.metrics import auc, roc_curve, roc_auc_score, r2_score, mean_squared_error, mean_absolute_error, classification_report
from sklearn.model_selection import KFold, StratifiedKFold, cross_validate, cross_val_score, GridSearchCV
from sklearn.linear_model import LogisticRegression, Ridge, LinearRegression
from mlens.ensemble import SuperLearner
from sklearn.ensemble import RandomForestRegressor 

from catboost import CatBoostRegressor
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from jupyterthemes import jtplot
from pylab import *
from geopy.distance import geodesic, great_circle
pd.options.display.max_rows = 999
%matplotlib inline
jtplot.style()
from scipy.stats import t
from scipy.stats import norm
from scipy import stats
from geopy.distance import geodesic

In [1866]:
# в первую очередь обозначим широту и олготу Кремля, чтобы была доп фича - от расстояния до него зависит стоимость комнаты
data = pd.read_csv("./rooms_df.csv")
kreml_lat = 55.75167
kreml_lng = 37.61778
data.head(2)

,itemId,commission,"room_area, m2",rooms_count,floor,floors_count,house_type,lat,lng,colors,...,metro_distance,station_1,station_2,station_3,metro_distance_1,metro_distance_2,metro_distance_3,name,address,"price, rub"
0,2092032155,Собственник,18.0,1,4,5,Панельный,55.752503,37.604400,"['#0072BA', '#1EBCEF']",...,"{'18': 0.4, '4': 0.3, '6': 0}",18.0,4.0,6.0,0.4,0.3,0.0,Арбатская,"Москва, улица Воздвиженка, 9",850000
1,2068911692,Посредник,23.0,3,3,5,Кирпичный,55.748376,37.599256,['#E42313'],...,"{'18': 0.7, '52': 0.5, '6': 0.6}",18.0,52.0,6.0,0.7,0.5,0.6,Кропоткинская,"Москва, Гоголевский бульвар, 25с1",11500000


### нашла доп данные в поисковиках - рейтинг жилья по станциям метро  - добавим из него стоимость квартир и стоимость квадратного метра


In [1867]:
ra = pd.read_csv("./raiting.csv", delimiter = '\t', header=None)[6:]
ra = ra.to_dict()[0]

In [1868]:
# index = []
metro_name = []
price_m2 = []
price_room_m2 = []
average_flat_price = []
values = []
for key, value in ra.items():
    values.append(value)
for idx, v in enumerate(values):
    if len(str(v)) <= 3:
#         index.append(v)
        metro_name.append(values[idx+1])
        price_m2.append(values[idx+2])
#         delta_room_price.append(values[idx+3])
        average_flat_price.append(int(values[idx+4]))
for i in price_m2:
    price = int(i.replace(" ", ""))
    price_room_m2.append(price)
rating_dict = {
    'name': metro_name, 'price_room_m2': price_room_m2, 'average_flat_price': average_flat_price
}
rating = pd.DataFrame.from_dict(rating_dict)
rating.head()

,name,price_room_m2,average_flat_price
0,Кропоткинская,492846,36126805
1,Охотный ряд,472369,35768476
2,Площадь Революции,472369,35768476
3,Театральная,472369,35768476
4,Пушкинская,470822,33025865


### также добавим с этого же сайта id по каждому метро и координаты метро

In [1869]:
codes = [{"id":1,"name":"Авиамоторная"},{"id":2,"name":"Автозаводская"},{"id":3,"name":"Академическая"},{"id":4,"name":"Александровский сад"},{"id":151,"name":"Алексеевская"},{"id":2135,"name":"Алма-Атинская"},{"id":5,"name":"Алтуфьево"},{"id":2174,"name":"Андроновка"},{"id":148,"name":"Аннино"},{"id":6,"name":"Арбатская"},{"id":7,"name":"Аэропорт"},{"id":8,"name":"Бабушкинская"},{"id":9,"name":"Багратионовская"},{"id":2159,"name":"Балтийская"},{"id":10,"name":"Баррикадная"},{"id":11,"name":"Бауманская"},{"id":12,"name":"Беговая"},{"id":2179,"name":"Белокаменная"},{"id":2207,"name":"Беломорская"},{"id":13,"name":"Белорусская"},{"id":14,"name":"Беляево"},{"id":15,"name":"Бибирево"},{"id":16,"name":"Библиотека им. Ленина"},{"id":2145,"name":"Битцевский парк"},{"id":215,"name":"Борисово"},{"id":18,"name":"Боровицкая"},{"id":2202,"name":"Боровское шоссе"},{"id":19,"name":"Ботанический сад"},{"id":20,"name":"Братиславская"},{"id":1010,"name":"Бульвар адмирала Ушакова"},{"id":149,"name":"Бульвар Дмитрия Донского"},{"id":127,"name":"Бульвар Рокоссовского"},{"id":1012,"name":"Бунинская аллея"},{"id":2155,"name":"Бутырская"},{"id":22,"name":"Варшавская"},{"id":21,"name":"ВДНХ"},{"id":2168,"name":"Верхние котлы"},{"id":2190,"name":"Верхние Лихоборы"},{"id":23,"name":"Владыкино"},{"id":24,"name":"Водный стадион"},{"id":25,"name":"Войковская"},{"id":26,"name":"Волгоградский проспект"},{"id":27,"name":"Волжская"},{"id":1003,"name":"Волоколамская"},{"id":28,"name":"Воробьевы Горы"},{"id":152,"name":"Выставочная"},{"id":29,"name":"Выхино"},{"id":2200,"name":"Говорово"},{"id":2146,"name":"Деловой центр"},{"id":2163,"name":"Деловой центр (МЦК)"},{"id":30,"name":"Динамо"},{"id":31,"name":"Дмитровская"},{"id":32,"name":"Добрынинская"},{"id":33,"name":"Домодедовская"},{"id":2001,"name":"Достоевская"},{"id":34,"name":"Дубровка"},{"id":2143,"name":"Жулебино"},{"id":2169,"name":"ЗИЛ"},{"id":2184,"name":"Зорге"},{"id":217,"name":"Зябликово"},{"id":2176,"name":"Измайлово"},{"id":35,"name":"Измайловская"},{"id":37,"name":"Калужская"},{"id":38,"name":"Кантемировская"},{"id":39,"name":"Каховская"},{"id":40,"name":"Каширская"},{"id":41,"name":"Киевская"},{"id":42,"name":"Китай-город"},{"id":43,"name":"Кожуховская"},{"id":44,"name":"Коломенская"},{"id":2215,"name":"Коммунарка"},{"id":45,"name":"Комсомольская"},{"id":46,"name":"Коньково"},{"id":2182,"name":"Коптево"},{"id":2211,"name":"Косино"},{"id":2151,"name":"Котельники"},{"id":47,"name":"Красногвардейская"},{"id":48,"name":"Краснопресненская"},{"id":49,"name":"Красносельская"},{"id":50,"name":"Красные ворота"},{"id":51,"name":"Крестьянская застава"},{"id":52,"name":"Кропоткинская"},{"id":53,"name":"Крылатское"},{"id":2167,"name":"Крымская"},{"id":54,"name":"Кузнецкий мост"},{"id":55,"name":"Кузьминки"},{"id":56,"name":"Кунцевская"},{"id":57,"name":"Курская"},{"id":58,"name":"Кутузовская"},{"id":59,"name":"Ленинский проспект"},{"id":2142,"name":"Лермонтовский проспект"},{"id":2144,"name":"Лесопарковая"},{"id":2222,"name":"Лефортово"},{"id":2158,"name":"Лихоборы"},{"id":2177,"name":"Локомотив"},{"id":2186,"name":"Ломоносовский проспект"},{"id":60,"name":"Лубянка"},{"id":2165,"name":"Лужники"},{"id":2209,"name":"Лухмановская"},{"id":61,"name":"Люблино"},{"id":62,"name":"Марксистская"},{"id":2002,"name":"Марьина Роща"},{"id":63,"name":"Марьино"},{"id":64,"name":"Маяковская"},{"id":65,"name":"Медведково"},{"id":1004,"name":"Международная"},{"id":66,"name":"Менделеевская"},{"id":2185,"name":"Минская"},{"id":1001,"name":"Митино"},{"id":2205,"name":"Мичуринский проспект"},{"id":67,"name":"Молодежная"},{"id":1002,"name":"Мякинино"},{"id":68,"name":"Нагатинская"},{"id":69,"name":"Нагорная"},{"id":70,"name":"Нахимовский проспект"},{"id":2208,"name":"Некрасовка"},{"id":2173,"name":"Нижегородская"},{"id":71,"name":"Новогиреево"},{"id":2133,"name":"Новокосино"},{"id":72,"name":"Новокузнецкая"},{"id":2203,"name":"Новопеределкино"},{"id":73,"name":"Новослободская"},{"id":2172,"name":"Новохохловская"},{"id":17,"name":"Новоясеневская"},{"id":74,"name":"Новые Черемушки"},{"id":2199,"name":"Озёрная"},{"id":2157,"name":"Окружная"},{"id":2220,"name":"Окская"},{"id":75,"name":"Октябрьская"},{"id":76,"name":"Октябрьское поле"},{"id":2214,"name":"Ольховая"},{"id":77,"name":"Орехово"},{"id":78,"name":"Отрадное"},{"id":79,"name":"Охотный ряд"},{"id":80,"name":"Павелецкая"},{"id":2183,"name":"Панфиловская"},{"id":82,"name":"Парк культуры"},{"id":81,"name":"Парк Победы"},{"id":36,"name":"Партизанская"},{"id":83,"name":"Первомайская"},{"id":84,"name":"Перово"},{"id":2195,"name":"Петровский парк"},{"id":85,"name":"Петровско-Разумовская"},{"id":86,"name":"Печатники"},{"id":87,"name":"Пионерская"},{"id":88,"name":"Планерная"},{"id":2166,"name":"Площадь Гагарина"},{"id":89,"name":"Площадь Ильича"},{"id":90,"name":"Площадь революции"},{"id":91,"name":"Полежаевская"},{"id":92,"name":"Полянка"},{"id":93,"name":"Пражская"},{"id":94,"name":"Преображенская площадь"},{"id":2213,"name":"Прокшино"},{"id":95,"name":"Пролетарская"},{"id":96,"name":"Проспект Вернадского"},{"id":97,"name":"Проспект Мира"},{"id":98,"name":"Профсоюзная"},{"id":99,"name":"Пушкинская"},{"id":2136,"name":"Пятницкое шоссе"},{"id":2187,"name":"Раменки"},{"id":2204,"name":"Рассказовка"},{"id":100,"name":"Речной вокзал"},{"id":101,"name":"Рижская"},{"id":102,"name":"Римская"},{"id":2180,"name":"Ростокино"},{"id":2149,"name":"Румянцево"},{"id":103,"name":"Рязанский проспект"},{"id":104,"name":"Савеловская"},{"id":2150,"name":"Саларьево"},{"id":105,"name":"Свиблово"},{"id":106,"name":"Севастопольская"},{"id":2191,"name":"Селигерская"},{"id":107,"name":"Семеновская"},{"id":108,"name":"Серпуховская"},{"id":1005,"name":"Славянский бульвар"},{"id":109,"name":"Смоленская"},{"id":110,"name":"Сокол"},{"id":2181,"name":"Соколиная гора"},{"id":111,"name":"Сокольники"},{"id":2201,"name":"Солнцево"},{"id":2147,"name":"Спартак"},{"id":112,"name":"Спортивная"},{"id":1007,"name":"Сретенский бульвар"},{"id":2221,"name":"Стахановская"},{"id":2160,"name":"Стрешнево"},{"id":214,"name":"Строгино"},{"id":113,"name":"Студенческая"},{"id":114,"name":"Сухаревская"},{"id":115,"name":"Сходненская"},{"id":116,"name":"Таганская"},{"id":117,"name":"Тверская"},{"id":118,"name":"Театральная"},{"id":119,"name":"Текстильщики"},{"id":120,"name":"Теплый стан"},{"id":2152,"name":"Технопарк"},{"id":121,"name":"Тимирязевская"},{"id":122,"name":"Третьяковская"},{"id":2148,"name":"Тропарево"},{"id":1006,"name":"Трубная"},{"id":123,"name":"Тульская"},{"id":124,"name":"Тургеневская"},{"id":125,"name":"Тушинская"},{"id":2171,"name":"Угрешская"},{"id":126,"name":"Улица 1905 года"},{"id":128,"name":"Улица академика Янгеля"},{"id":1011,"name":"Улица Горчакова"},{"id":2210,"name":"Улица Дмитриевского"},{"id":1009,"name":"Улица Скобелевская"},{"id":1008,"name":"Улица Старокачаловская"},{"id":129,"name":"Университет"},{"id":2212,"name":"Филатов луг"},{"id":130,"name":"Филевский парк"},{"id":131,"name":"Фили"},{"id":2154,"name":"Фонвизинская"},{"id":132,"name":"Фрунзенская"},{"id":2188,"name":"Ховрино"},{"id":2161,"name":"Хорошево"},{"id":2193,"name":"Хорошевская"},{"id":133,"name":"Царицыно"},{"id":134,"name":"Цветной бульвар"},{"id":2194,"name":"ЦСКА"},{"id":135,"name":"Черкизовская"},{"id":136,"name":"Чертановская"},{"id":137,"name":"Чеховская"},{"id":138,"name":"Чистые пруды"},{"id":139,"name":"Чкаловская"},{"id":140,"name":"Шаболовская"},{"id":2162,"name":"Шелепиха"},{"id":216,"name":"Шипиловская"},{"id":141,"name":"Шоссе Энтузиастов"},{"id":142,"name":"Щелковская"},{"id":143,"name":"Щукинская"},{"id":144,"name":"Электрозаводская"},{"id":2219,"name":"Юго-Восточная"},{"id":145,"name":"Юго-Западная"},{"id":146,"name":"Южная"},{"id":147,"name":"Ясенево"}]
metro_codes = pd.DataFrame.from_dict(codes)
df_metro_lat_lng = pd.read_csv("./GPS_code_metro.csv",  delimiter = '\t', names=['name','lat_metro','lng_metro'])                
df_metro_lat_lng['name'] = df_metro_lat_lng['name'].apply(lambda x: x[:-7] if x[-6:] == '(стр.)' else x)

metro_codes.info(), df_metro_lat_lng.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 238 entries, 0 to 237
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      238 non-null    int64 
 1   name    238 non-null    object
dtypes: int64(1), object(1)
memory usage: 3.8+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 165 entries, 0 to 164
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   name       165 non-null    object 
 1   lat_metro  165 non-null    float64
 2   lng_metro  165 non-null    float64
dtypes: float64(2), object(1)
memory usage: 4.0+ KB


(None, None)

In [1870]:
metro_data1 = pd.merge(df_metro_lat_lng, metro_codes, how='inner', on="name")
metro_data = pd.merge(rating, metro_data1, how='inner', on="name")


metro_data[metro_data['id'].isnull()].head(3)
metro_data.isnull().sum()
metro_data.head()

,name,price_room_m2,average_flat_price,lat_metro,lng_metro,id
0,Кропоткинская,492846,36126805,55.74525,37.60463,52
1,Охотный ряд,472369,35768476,55.75703,37.61614,79
2,Театральная,472369,35768476,55.75857,37.61770,118
3,Пушкинская,470822,33025865,55.76565,37.60417,99
4,Тверская,470822,33025865,55.76520,37.60352,117


In [1871]:
df = pd.merge(data, metro_data, how='inner', on="name")

### в парсинге были фичи, где в наличие 3 ближайших метро и расстояния до них, но очень много пустых  значений - они нам не подходят, - оставляем только однометро и координаты метро, комнаты, Кремля.  

In [1872]:
#там, где метро не достает для дальнейшего расчета, поставим общее метро id
df.loc[df['station_1'].isnull(), 'station_1'] = df['id']
df.loc[df['station_2'].isnull(), 'station_2'] = df['id']
df.loc[df['station_3'].isnull(), 'station_3'] = df['id']

In [1873]:
# для расчета растояний нам понадобятся данные дельты широты и долготы между метро и комнатой
df['coords_room'] = list(zip([lat for lat in df['lat']], [lng for lng in df['lng']]))
df['coords_metro'] = list(zip([lat_metro for lat_metro in df['lat_metro']], 
                                  [lng_metro for lng_metro in df['lng_metro']]))


In [1874]:
# расстояния до метро
for idx, i in enumerate(df['coords_room']):   
    df[['distance_to_metro']] = (geodesic(df['coords_room'][idx], df['coords_metro'][idx])).km
    
# расстояния до Кремля
for idx, i in enumerate(df['coords_room']):   
    df[['distance_to_kreml']] = (geodesic(df['coords_room'][idx], (kreml_lat, kreml_lng))).km
    
    
#ВСЕ, ЧТО ВЫШЕ, НЕВЕРНО
for idx, i in enumerate(df['coords_room']):   
    df['distance_to_metro'][idx] = (geodesic(df['coords_room'][idx], df['coords_metro'][idx])).km
    
# расстояния до Кремля
for idx, i in enumerate(df['coords_room']):   
    df['distance_to_kreml'][idx] = (geodesic(df['coords_room'][idx], (kreml_lat, kreml_lng))).km
     
        

/Users/natalaurina/.pyenv/versions/3.7.8/envs/jupyter_notebook/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/Users/natalaurina/.pyenv/versions/3.7.8/envs/jupyter_notebook/lib/python3.7/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()


In [1875]:
df[209:210][['station_1', 'metro_distance_1', 'station_2', 'metro_distance_2', 'station_3', 'metro_distance_3', 'distance_to_metro', 'id', 'name']]
df.head()

,itemId,commission,"room_area, m2",rooms_count,floor,floors_count,house_type,lat,lng,colors,...,"price, rub",price_room_m2,average_flat_price,lat_metro,lng_metro,id,coords_room,coords_metro,distance_to_metro,distance_to_kreml
0,2092032155,Собственник,18.0,1,4,5,Панельный,55.752503,37.604400,"['#0072BA', '#1EBCEF']",...,850000,469531,35226082,55.75228,37.60357,6,"(55.7525032514771, 37.604400160747)","(55.752280000000006, 37.60357)",0.057750,0.845248
1,2068911692,Посредник,23.0,3,3,5,Кирпичный,55.748376,37.599256,['#E42313'],...,11500000,492846,36126805,55.74525,37.60463,52,"(55.748376, 37.599256)","(55.74525, 37.60463)",0.484799,1.219658
2,2105735154,Посредник,43.0,2,1,5,Кирпичный,55.757995,37.644970,['#E42313'],...,350000,375962,21840385,55.76426,37.63890,138,"(55.757995232753295, 37.6449703160173)","(55.76426, 37.6389)",0.794815,1.846751
3,2107727719,Собственник,16.0,2,5,16,Кирпичный,55.764632,37.582592,['#943E90'],...,3700000,355842,19850364,55.76027,37.58111,10,"(55.76463199999999, 37.582592)","(55.76027, 37.581109999999995)",0.494485,2.638772
4,2067107338,Посредник,19.0,3,5,5,Кирпичный,55.734658,37.630751,"['#4FB04F', '#915133']",...,6150000,318625,18027149,55.73130,37.63612,80,"(55.734657999999996, 37.630751000000004)","(55.7313, 37.63612)",0.503532,2.061833


### итого получилась таблица со значениями втом числе расстояний до Кремля и метро, но лишних столбцов много - оставляем только необходимое

In [1876]:
# удалим лишние столбцы, чтобы не мешали
df1 = df.drop(['lat', 'lng', 'address', 'name', 'lat_metro', 'lng_metro',
        'coords_room', 'coords_metro', 'url', 'metro_distance', 'itemId',
        'station_1', 'station_2', 'station_3', 'metro_distance_1', 'metro_distance_2',
        'metro_distance_3', 'colors'
              ], axis=1)
# data = data_X.drop(['Item_Outlet_Sales', 'Name_Item_Identifier'], axis=1)
df1.head()

,commission,"room_area, m2",rooms_count,floor,floors_count,house_type,"price, rub",price_room_m2,average_flat_price,id,distance_to_metro,distance_to_kreml
0,Собственник,18.0,1,4,5,Панельный,850000,469531,35226082,6,0.057750,0.845248
1,Посредник,23.0,3,3,5,Кирпичный,11500000,492846,36126805,52,0.484799,1.219658
2,Посредник,43.0,2,1,5,Кирпичный,350000,375962,21840385,138,0.794815,1.846751
3,Собственник,16.0,2,5,16,Кирпичный,3700000,355842,19850364,10,0.494485,2.638772
4,Посредник,19.0,3,5,5,Кирпичный,6150000,318625,18027149,80,0.503532,2.061833


In [1877]:
# df1.loc[df1['metro_distance_1'].isnull(), 'metro_distance_1'] = df['distance_to_metro']
# df1.loc[df1['metro_distance_2'].isnull(), 'metro_distance_2'] = df['distance_to_metro']
# df1.loc[df1['metro_distance_3'].isnull(), 'metro_distance_3'] = df['distance_to_metro']
# df1.loc[df1['station_1'].isnull(), 'station_1'] = df['id']
# df1.loc[df1['station_2'].isnull(), 'station_2'] = df['id']
# df1.loc[df1['station_3'].isnull(), 'station_3'] = df['id']


# df1[df1['station_3'].isnull()]


### Удаляем шумы

In [1878]:
# УДАЛИТЬ ШУМЫ!
df1 = df1.drop(df1[df1['rooms_count'] == 1].index)
df1 = df1.drop(df1[df1['price, rub']< 2000000].index)
# df1 = df1.drop(df1[df1['floors_count'] == 5].index)
df1 = df1.drop(df1[df1['room_area, m2'] < 8].index)
# df1 = df1.drop(df1[df1['floors_count'] < 6].index)
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 430 entries, 1 to 469
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   commission          430 non-null    object 
 1   room_area, m2       430 non-null    float64
 2   rooms_count         430 non-null    int64  
 3   floor               430 non-null    int64  
 4   floors_count        430 non-null    int64  
 5   house_type          430 non-null    object 
 6   price, rub          430 non-null    int64  
 7   price_room_m2       430 non-null    int64  
 8   average_flat_price  430 non-null    int64  
 9   id                  430 non-null    int64  
 10  distance_to_metro   430 non-null    float64
 11  distance_to_kreml   430 non-null    float64
dtypes: float64(3), int64(7), object(2)
memory usage: 43.7+ KB


In [1880]:
df1.head(1)
df1['price, rub'].value_counts().sum()
df1.info()

y = df1['price, rub']
data = df1.drop(['price, rub'], axis=1)
X = pd.get_dummies(data, columns=['commission', 'house_type', 'id'
                                  #,'station_1', 'station_2', 'station_3', 
                                 #'metro_distance_1', 'metro_distance_2', 'metro_distance_3'
                                 ])
X_train, X_test, y_train, y_test = train_test_split(X,y)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 430 entries, 1 to 469
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   commission          430 non-null    object 
 1   room_area, m2       430 non-null    float64
 2   rooms_count         430 non-null    int64  
 3   floor               430 non-null    int64  
 4   floors_count        430 non-null    int64  
 5   house_type          430 non-null    object 
 6   price, rub          430 non-null    int64  
 7   price_room_m2       430 non-null    int64  
 8   average_flat_price  430 non-null    int64  
 9   id                  430 non-null    int64  
 10  distance_to_metro   430 non-null    float64
 11  distance_to_kreml   430 non-null    float64
dtypes: float64(3), int64(7), object(2)
memory usage: 43.7+ KB


In [1881]:
import numpy as np
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
X_train_scaled = ss.fit_transform(X_train)
X_test_scaled = ss.transform(X_test)
y_train = np.array(y_train)

/Users/natalaurina/.pyenv/versions/3.7.8/envs/jupyter_notebook/lib/python3.7/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Users/natalaurina/.pyenv/versions/3.7.8/envs/jupyter_notebook/lib/python3.7/site-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/Users/natalaurina/.pyenv/versions/3.7.8/envs/jupyter_notebook/lib/python3.7/site-packages/ipykernel_launcher.py:5: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  """


In [1882]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import recall_score
rfc = RandomForestClassifier()
rfc.fit(X_train_scaled, y_train)
display(rfc.score(X_train_scaled, y_train))
display(rfc.score(X_test_scaled, y_test))


/Users/natalaurina/.pyenv/versions/3.7.8/envs/jupyter_notebook/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.9875776397515528

0.046296296296296294

In [1883]:
from sklearn.ensemble import RandomForestRegressor
model_forest = RandomForestRegressor()
model_forest.fit(X_train, y_train)
y_pred_test = model_forest.predict(X_test)
y_pred_train = model_forest.predict(X_train)
print(mean_squared_error(y_train, y_pred_train), mean_squared_error(y_test, y_pred_test))
print(r2_score(y_train, y_pred_train), r2_score(y_test, y_pred_test)  )

336269248016.5238 2713964559445.932
0.887191016011691 0.33566724399812964


/Users/natalaurina/.pyenv/versions/3.7.8/envs/jupyter_notebook/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [1884]:
model_cat = CatBoostRegressor()
model_cat.fit(X_train, y_train)
y_pred_test = model_cat.predict(X_test)
y_pred_train = model_cat.predict(X_train)
# print(mean_squared_error(y_train, y_pred_train), mean_squared_error(y_test, y_pred_test))
r2_score(y_train, y_pred_train), r2_score(y_test, y_pred_test)

Learning rate set to 0.031901
0:	learn: 1705738.3543166	total: 2.86ms	remaining: 2.86s
1:	learn: 1686028.9509215	total: 4.76ms	remaining: 2.38s
2:	learn: 1666605.0503500	total: 8.2ms	remaining: 2.72s
3:	learn: 1650282.6656401	total: 10.7ms	remaining: 2.66s
4:	learn: 1632386.8871454	total: 13.1ms	remaining: 2.61s
5:	learn: 1615539.3666720	total: 15.4ms	remaining: 2.55s
6:	learn: 1599235.7293256	total: 24.6ms	remaining: 3.49s
7:	learn: 1582334.3078603	total: 28.6ms	remaining: 3.54s
8:	learn: 1563974.2695118	total: 32.7ms	remaining: 3.6s
9:	learn: 1549073.8005962	total: 35.1ms	remaining: 3.47s
10:	learn: 1532812.8623337	total: 38.6ms	remaining: 3.47s
11:	learn: 1516855.6675121	total: 40.4ms	remaining: 3.32s
12:	learn: 1503880.5148024	total: 42.2ms	remaining: 3.2s
13:	learn: 1490779.6752836	total: 44.1ms	remaining: 3.11s
14:	learn: 1477516.6812030	total: 46ms	remaining: 3.02s
15:	learn: 1464750.1072906	total: 47.7ms	remaining: 2.93s
16:	learn: 1453388.3299825	total: 49.6ms	remaining: 2.87s

174:	learn: 863547.8576931	total: 354ms	remaining: 1.67s
175:	learn: 861191.2407723	total: 357ms	remaining: 1.67s
176:	learn: 859961.3486562	total: 360ms	remaining: 1.67s
177:	learn: 859370.7659861	total: 363ms	remaining: 1.67s
178:	learn: 856487.2712535	total: 365ms	remaining: 1.68s
179:	learn: 854725.0599729	total: 368ms	remaining: 1.68s
180:	learn: 852840.0720718	total: 370ms	remaining: 1.67s
181:	learn: 851464.6562609	total: 373ms	remaining: 1.68s
182:	learn: 850400.5066287	total: 376ms	remaining: 1.68s
183:	learn: 849679.2784149	total: 378ms	remaining: 1.68s
184:	learn: 849161.0318719	total: 381ms	remaining: 1.68s
185:	learn: 847704.9357685	total: 383ms	remaining: 1.68s
186:	learn: 846255.6599834	total: 386ms	remaining: 1.68s
187:	learn: 843871.0046209	total: 388ms	remaining: 1.68s
188:	learn: 843087.7009896	total: 391ms	remaining: 1.68s
189:	learn: 840526.5942167	total: 402ms	remaining: 1.71s
190:	learn: 839672.5416376	total: 404ms	remaining: 1.71s
191:	learn: 838784.6091168	tota

329:	learn: 712525.7662575	total: 705ms	remaining: 1.43s
330:	learn: 711818.6435374	total: 708ms	remaining: 1.43s
331:	learn: 711022.1538223	total: 711ms	remaining: 1.43s
332:	learn: 710770.6562775	total: 713ms	remaining: 1.43s
333:	learn: 709072.0368384	total: 715ms	remaining: 1.43s
334:	learn: 707173.1532821	total: 717ms	remaining: 1.42s
335:	learn: 706527.1701718	total: 720ms	remaining: 1.42s
336:	learn: 706243.6090910	total: 723ms	remaining: 1.42s
337:	learn: 705965.1050347	total: 725ms	remaining: 1.42s
338:	learn: 705663.2585702	total: 728ms	remaining: 1.42s
339:	learn: 705427.7659572	total: 730ms	remaining: 1.42s
340:	learn: 704201.2928543	total: 732ms	remaining: 1.41s
341:	learn: 703923.5403226	total: 733ms	remaining: 1.41s
342:	learn: 703694.2493969	total: 735ms	remaining: 1.41s
343:	learn: 703101.6778249	total: 738ms	remaining: 1.41s
344:	learn: 702865.5067659	total: 740ms	remaining: 1.4s
345:	learn: 702597.4832660	total: 741ms	remaining: 1.4s
346:	learn: 701980.5243784	total:

499:	learn: 607863.1017090	total: 1.06s	remaining: 1.06s
500:	learn: 607736.7769174	total: 1.06s	remaining: 1.06s
501:	learn: 605580.7089956	total: 1.07s	remaining: 1.06s
502:	learn: 604703.4806306	total: 1.07s	remaining: 1.06s
503:	learn: 604574.2572746	total: 1.07s	remaining: 1.06s
504:	learn: 604451.8077889	total: 1.08s	remaining: 1.05s
505:	learn: 604344.1554238	total: 1.08s	remaining: 1.05s
506:	learn: 603449.2519405	total: 1.08s	remaining: 1.05s
507:	learn: 602851.3053510	total: 1.08s	remaining: 1.05s
508:	learn: 601873.2669375	total: 1.09s	remaining: 1.05s
509:	learn: 601740.9599860	total: 1.09s	remaining: 1.05s
510:	learn: 601614.9825750	total: 1.09s	remaining: 1.04s
511:	learn: 600821.7967757	total: 1.09s	remaining: 1.04s
512:	learn: 600429.5246778	total: 1.1s	remaining: 1.04s
513:	learn: 600134.6834512	total: 1.1s	remaining: 1.04s
514:	learn: 598632.4132771	total: 1.1s	remaining: 1.04s
515:	learn: 597864.6335358	total: 1.1s	remaining: 1.04s
516:	learn: 596194.7017386	total: 1

671:	learn: 508817.0126182	total: 1.42s	remaining: 691ms
672:	learn: 507952.1083536	total: 1.42s	remaining: 689ms
673:	learn: 507629.1591908	total: 1.42s	remaining: 688ms
674:	learn: 507028.4988599	total: 1.42s	remaining: 685ms
675:	learn: 506519.2137089	total: 1.43s	remaining: 684ms
676:	learn: 506114.4866253	total: 1.43s	remaining: 682ms
677:	learn: 506027.7721974	total: 1.43s	remaining: 680ms
678:	learn: 505933.1192263	total: 1.43s	remaining: 678ms
679:	learn: 505829.9766041	total: 1.44s	remaining: 676ms
680:	learn: 504768.4002560	total: 1.44s	remaining: 674ms
681:	learn: 504421.4577986	total: 1.44s	remaining: 671ms
682:	learn: 504315.4585765	total: 1.44s	remaining: 669ms
683:	learn: 503867.1819394	total: 1.44s	remaining: 667ms
684:	learn: 503718.0676509	total: 1.45s	remaining: 665ms
685:	learn: 503422.1234695	total: 1.45s	remaining: 663ms
686:	learn: 502976.8362028	total: 1.45s	remaining: 661ms
687:	learn: 501686.6176989	total: 1.45s	remaining: 659ms
688:	learn: 501529.4443970	tota

848:	learn: 434861.0967810	total: 1.77s	remaining: 315ms
849:	learn: 433934.8934897	total: 1.77s	remaining: 313ms
850:	learn: 432806.4321435	total: 1.78s	remaining: 311ms
851:	learn: 432066.6813600	total: 1.78s	remaining: 309ms
852:	learn: 431607.3649690	total: 1.78s	remaining: 307ms
853:	learn: 431547.0788701	total: 1.78s	remaining: 305ms
854:	learn: 431212.3824312	total: 1.79s	remaining: 303ms
855:	learn: 430710.7159119	total: 1.79s	remaining: 301ms
856:	learn: 429050.0296236	total: 1.79s	remaining: 299ms
857:	learn: 428996.5601417	total: 1.79s	remaining: 297ms
858:	learn: 428675.5834892	total: 1.8s	remaining: 295ms
859:	learn: 428615.4716449	total: 1.8s	remaining: 293ms
860:	learn: 428436.1862216	total: 1.8s	remaining: 291ms
861:	learn: 427982.6212804	total: 1.81s	remaining: 289ms
862:	learn: 427095.7197701	total: 1.81s	remaining: 287ms
863:	learn: 426445.4209346	total: 1.81s	remaining: 285ms
864:	learn: 426230.5833641	total: 1.81s	remaining: 283ms
865:	learn: 425673.3558181	total: 

994:	learn: 380470.7794945	total: 2.12s	remaining: 10.7ms
995:	learn: 380351.9801992	total: 2.12s	remaining: 8.53ms
996:	learn: 379936.7952310	total: 2.12s	remaining: 6.39ms
997:	learn: 379881.1908162	total: 2.13s	remaining: 4.27ms
998:	learn: 379576.8064335	total: 2.13s	remaining: 2.14ms
999:	learn: 379531.0465541	total: 2.14s	remaining: 0us


(0.9516773045722551, 0.3157378409618242)

In [1885]:
#Бустинг
model_cat = CatBoostRegressor(n_estimators=1000, learning_rate=0.2, max_depth=4, silent=True)
model_cat.fit(X_train, y_train)
y_pred_test = model_cat.predict(X_test)
y_pred_train = model_cat.predict(X_train)
print(mean_squared_error(y_train, y_pred_train), mean_squared_error(y_test, y_pred_test))
print(r2_score(y_train, y_pred_train), r2_score(y_test, y_pred_test)  )   

6815733318.516409 2765795046973.1807
0.997713510957864 0.32298001471797766


In [1886]:
ensemble = SuperLearner()

# Build the first layer
ensemble.add([RandomForestRegressor(n_estimators=500, max_features=15), Ridge(alpha=1.0)])

# Attach the final meta estimator
ensemble.add_meta(LinearRegression())
ensemble.fit(X_train, y_train)

# Predict
preds_train, preds_test = ensemble.predict(X_train), ensemble.predict(X_test)
r2_score(y_train, preds_train), r2_score(y_test, preds_test)

(0.9235829660515876, 0.3858317980282514)

In [1893]:
# skf = KFold(n_splits=10, random_state=None, shuffle=False)
# train_metric, test_metric = [], []
# for train_index, test_index in skf.split(X, y):
#     X_train, X_test = X.loc[train_index], X.loc[test_index]
#     y_train, y_test = y.loc[train_index], y.loc[test_index]
#     cat_model = CatBoostRegressor(n_estimators=1000, learning_rate=0.2, max_depth=4, silent=True)
#     cat_model.fit(X_train, y_train)
#     y_pred_train, y_pred_test = clf_tree.predict(X_train), clf_tree.predict(X_test)
#     mean_squared_error(y_train, y_pred_train), mean_squared_error(y_test, y_pred_test)
#     train_metric.append(r2_score(y_train, y_pred_train))
#     test_metric.append(r2_score(y_test, y_pred_test))
#     print(r2_score(y_train, y_pred_train), r2_score(y_test, y_pred_test))
# print(sum(train_metric)/len(train_metric))
# print(sum(test_metric)/len(test_metric))

In [1888]:
#проверим важность фич на основании построенной  модели
# imp = pd.Series(ensemble.feature_importances_)
# imp.sort_values(ascending=False).head(7)

In [1889]:
#проверим важность фич на основании построенной  модели
# imp = pd.Series(ensemble.feature_importances_)
# imp.sort_values(ascending=False).head(8)

In [1890]:
X.head()

,"room_area, m2",rooms_count,floor,floors_count,price_room_m2,average_flat_price,distance_to_metro,distance_to_kreml,commission_Посредник,commission_Собственник,...,id_144,id_146,id_147,id_148,id_151,id_1001,id_1003,id_2001,id_2133,id_2185
1,23.0,3,3,5,492846,36126805,0.484799,1.219658,1,0,...,0,0,0,0,0,0,0,0,0,0
3,16.0,2,5,16,355842,19850364,0.494485,2.638772,0,1,...,0,0,0,0,0,0,0,0,0,0
4,19.0,3,5,5,318625,18027149,0.503532,2.061833,1,0,...,0,0,0,0,0,0,0,0,0,0
5,16.5,3,2,5,318625,18027149,0.523142,2.254765,0,1,...,0,0,0,0,0,0,0,0,0,0
6,15.6,3,6,8,318625,18027149,0.313922,2.329889,1,0,...,0,0,0,0,0,0,0,0,0,0
